In [19]:
from module.prepare import *
from itertools import product
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import ParameterGrid

import gc
import os
import re
import math
import sys
from collections import Counter
import random
from itertools import islice
import time
import configparser
import json

import seaborn as sns
import matplotlib.pyplot as plt
import bokeh

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy as sp
from scipy import stats

import sklearn
from joblib import dump, load
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.manifold import *
import sklearn.tree as Tr 

import lightgbm as lgb

import optuna

from boruta import BorutaPy

from module.metrics import *

In [10]:
cv = 5
generalize_ratio = 1.0/cv
test_ratio = 1.0/cv
tuning_mode = False

if tuning_mode:
    cv = 1

In [11]:
import json
import re

'''
['NPInter10412','reRPI2825','RPI488','RPI2241','RPI1807','LPI43250','EVLncRNAs']
'''
params_1 = {
    0:[(5,5,0.99)],
    1:[(3,4,0.96)], # stage1
    2:[(4,6,0.96)],
    3:[(3,3,0.975),(6,3,0.93),(4,3,0.975)],
    4:[(4,3,0.96)],
    5:[(4,6,0.945)]
}



In [31]:
_dataid = 3
global_params = params_1[_dataid][1]
### get conf of dataid
protein_k = global_params[0]
rna_k = global_params[1]
top_ratio = global_params[2]
### read data
[data,T] = ReadData(_dataid,protein_k,rna_k)
[X,Y] = ToMatrix(data,'dense')
### split dataset
[X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,0.2)
### dimensionality reduction
rf = RandomForestClassifier(n_jobs=-1,max_depth=5)
feature_selector = BorutaPy(rf, n_estimators=100, verbose=2, random_state=1,max_iter=1000)
feature_selector.fit(X_train, Y_train)

read data 6 3
=============RETRIEVE TRIAN DATA=================
# DEBUG: # DEBUG: **************new dl 3***************
# DEBUG: READ SEQ FROM FILE
# DEBUG: READ CLUSTER FROM FILE
ERROR:: regex  
ERROR:: regex  
# DEBUG: READ PAIR FROM FILE
# DEBUG: GENERATE NEGATIVE PAIR
# DEBUG: negative pair number 2240
INFO::count of negative pairs2240
# DEBUG: PAIR UNION
# DEBUG: EXTRACT FEATURES--PROTEIN
# DEBUG: EXTRACT FEATURES--RNA
# DEBUG: K-MER CALCULATION
# DEBUG: FEATURE UNION
# DEBUG: GARBAGE COLLECTION
MATRIX TRANSFORMATION
DEBUG:: total features count  29148
data shape 4480 29148
Iteration: 	1 / 1000
Confirmed: 	0
Tentative: 	29148
Rejected: 	0
Iteration: 	2 / 1000
Confirmed: 	0
Tentative: 	29148
Rejected: 	0
Iteration: 	3 / 1000
Confirmed: 	0
Tentative: 	29148
Rejected: 	0
Iteration: 	4 / 1000
Confirmed: 	0
Tentative: 	29148
Rejected: 	0
Iteration: 	5 / 1000
Confirmed: 	0
Tentative: 	29148
Rejected: 	0
Iteration: 	6 / 1000
Confirmed: 	0
Tentative: 	29148
Rejected: 	0
Iteration: 	7 / 10

BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                          class_weight=None, criterion='gini',
                                          max_depth=5, max_features='auto',
                                          max_leaf_nodes=None, max_samples=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=100, n_jobs=-1,
                                          oob_score=False,
                                          random_state=RandomState(MT19937) at 0x25F3B9476A8,
                                          verbose=0, warm_start=False),
         max_iter=1000, n_estimators=100,

In [26]:
X_train.shape

(3584, 59)

In [25]:
X_train = feature_selector.transform(X_train)
X_test = feature_selector.transform(X_test)

In [27]:
clf = lgb.LGBMClassifier(objective='cross_entropy', random_state=seed, n_jobs=6)
clf.fit(X_train,Y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=6, num_leaves=31,
               objective='cross_entropy', random_state=42, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [28]:
Ypred_train = clf.predict(X_train)
Ypred_test = clf.predict(X_test)

In [29]:
score_train = scoreFunction(Ypred_train,Y_train)
score_test = scoreFunction(Ypred_test,Y_test)

In [30]:
score_train,score_test

({'acc': 0.7748325892857143,
  'auc': 0.7761430691508826,
  'fpr': [0.0, 0.19021134593993325, 1.0],
  'tpr': [0.0, 0.7396416573348265, 1.0],
  'mcc': 0.5508563741541183,
  'tnr': 0.8097886540600667,
  'ppv': 0.7943475646422129,
  'f_score': 0.7660191359814438,
  'ap': 0.7172758527832781,
  'brier': 0.22516741071428573,
  'sensitivity': 0.7396416573348265},
 {'acc': 0.7176339285714286,
  'auc': 0.7205316112925875,
  'fpr': [0.0, 0.22850678733031674, 1.0],
  'tpr': [0.0, 0.6651982378854625, 1.0],
  'mcc': 0.4388718929907078,
  'tnr': 0.7714932126696833,
  'ppv': 0.749379652605459,
  'f_score': 0.704784130688448,
  'ap': 0.6681288815632286,
  'brier': 0.28236607142857145,
  'sensitivity': 0.6651982378854625})